In [0]:
#importação das bibliotecas
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql.functions import *
from functools import reduce
from pyspark.sql.window import Window
from datetime import datetime
from pyspark.sql.types import DoubleType
import pandas as pd

#instala a biblioteca para tirar caracter especial
!pip install unidecode
from unidecode import unidecode

     |████████████████████████████████| 235 kB 883 kB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-4655c5b9-97ce-4e6c-abfc-e36261f56698/bin/python -m pip install --upgrade pip' command.


##Funções Uteis

In [0]:
#função que trata o dataframe
def ajusta_layout_df(df: DataFrame, separador, pop_num) -> DataFrame:
  
  #pega o nome atual da coluna unica
  coluna = df.columns[0]

  #transforma em lista
  colunas_nome = unidecode(coluna).split(separador)
  colunas_nome.append('ajuste')

  df = df[coluna].str.split(separador, expand=True)
  
  #converte em spark
  df = spark.createDataFrame(df)

  #nome das colunas que são numeros
  colunas_num = df.columns

  #renomeia os campos
  for i in range(0, len(colunas_num)):
    df = df.withColumnRenamed(colunas_num[i], colunas_nome[i].lower().replace(' ', '_').strip())

  #remove o campo que geralmente vem quebrado
  colunas = df.columns
  coluna_filtro = colunas.pop(pop_num)

  #seleciona o layoyt final
  coluna_select_final = df.columns
  coluna_select_final.remove('ajuste')


  #ajusta o layout porque a coluna estado veio com dados faltando e o layout fica desconfigurado
  tmp = df.filter(col(coluna_filtro) == '').select([col for col in colunas])
  df = df.filter(col('ajuste').isNull()).select([col for col in coluna_select_final])
  df = df.union(tmp)
  
  return df

##Sobe os Dataframes

In [0]:
#definição dos diretorios
path_cep = 'https://raw.githubusercontent.com/igordeuzami/case_nestle/main/BaseCEP.csv'
path_cargos = 'https://raw.githubusercontent.com/igordeuzami/case_nestle/main/BaseCargos.csv'
path_clientes = 'https://raw.githubusercontent.com/igordeuzami/case_nestle/main/BaseClientes.csv'
path_funcionarios = 'https://raw.githubusercontent.com/igordeuzami/case_nestle/main/BaseFuncionarios.csv'
path_nivel = 'https://raw.githubusercontent.com/igordeuzami/case_nestle/main/BaseN%C3%ADvel.csv'
path_pq = 'https://raw.githubusercontent.com/igordeuzami/case_nestle/main/BasePQ.csv'

#sobe em dataframe e trata o dataframe
##########TRATAMENTO FEITO VIA FUNCAO
#base de CEP
cep = pd.read_csv(path_cep)
cep = ajusta_layout_df(cep, '|', 1)

#base de CARGOS
cargos = pd.read_csv(path_cargos)
cargos = ajusta_layout_df(cargos, ';', 1)


################################################
#tratamento manual no arquivo

#########base de FUNCIONARIOS
####para a base de funcionarios, foi substituido de forma manual o delimiter '||' por '|'
funcionarios = pd.read_csv(path_funcionarios,sep='|')
funcionarios = spark.createDataFrame(funcionarios)

colunas = funcionarios.columns

#renomeia os campos
for i in range(0, len(colunas)):
  funcionarios = funcionarios.withColumnRenamed(colunas[i], colunas[i].lower().replace(' ', '_').strip())

#########

#base de CLIENTES
clientes = pd.read_csv(path_clientes)

separador = ';'

#pega o nome atual da coluna unica
coluna = clientes.columns[0]

  #transforma em lista
colunas_nome = unidecode(coluna).split(separador)
colunas_nome.append('ajuste')

clientes = clientes[coluna].str.split(separador, expand=True)
  
#converte em spark
clientes = spark.createDataFrame(clientes)

#nome das colunas que são numeros
colunas_num = clientes.columns

  #renomeia os campos
for i in range(0, len(colunas_num)):
  clientes = clientes.withColumnRenamed(colunas_num[i], colunas_nome[i].lower().replace(' ', '_').strip())

#limpa base
clientes = clientes.filter(col('nivel_de_importancia').isin('1','2','3','4')).select('cliente',
            'valor_contrato_anual',
            'quantidade_de_servicos',
            'cargo_responsavel',
            'cep',
            'data_inicio_contrato',
            'nivel_de_importancia')



##Ajustes de formatos e limpeza nos dados

In [0]:
####ajuste campos
#cargos
cargos = cargos.withColumn('nivel', trim(regexp_replace('nivel', '[.,/@#!$%^&*";:{}=_`~()-]', ''))) \
                .withColumn('area', trim(regexp_replace('area', '[.,/#!@$%^&*";:{}=_`~()-]', ''))) \
                .withColumn('quadro', trim(regexp_replace('quadro', '[.,@/#!$%^&*";:{}=_`~()-]', '')))

####clientes
clientes = clientes.withColumn('cliente', trim('cliente')) \
            .withColumn('data_inicio_contrato', to_date('data_inicio_contrato', 'dd/MM/yyyy'))


###funcionarios
#trata os campos de valores 
funcionarios = funcionarios.withColumn('impostos', regexp_replace('impostos', ',' , '.' )) \
          .withColumn('beneficios', regexp_replace('beneficios', ',' , '.' )) \
          .withColumn('vt', regexp_replace('vt', ',' , '.' )) \
          .withColumn('vr', regexp_replace('vr', ',' , '.' ))

#converte para formato numero
funcionarios = funcionarios.withColumn('salario_base', funcionarios.impostos.cast(DoubleType())) \
                  .withColumn('impostos', funcionarios.impostos.cast(DoubleType())) \
                  .withColumn('beneficios', funcionarios.beneficios.cast(DoubleType())) \
                  .withColumn('vt', funcionarios.vt.cast(DoubleType())) \
                  .withColumn('vr', funcionarios.vr.cast(DoubleType()))

##Tratamentos de Dados Sensiveis

In [0]:
#########
#base de funcionarios
funcionarios = funcionarios.select('id_rh',
              'estado_civil',
              'data_de_nascimento',
              'cep',
              'data_de_contratacao',
              'data_de_demissao',
              'salario_base',
              'impostos',
              'beneficios',
              'vt',
              'vr',
              'cargo')



################################################
#base de CONTRATOS
#seleciona os clientes e valor de contrato distintos
contrato_id = clientes.select('cliente', 'valor_contrato_anual').distinct()

#adiciona o campo de id
contrato_id = contrato_id.withColumn('id_contrato', lit(1))
contrato_id = contrato_id.withColumn('id_contrato', rank().over(Window.partitionBy(col('id_contrato')).orderBy(col('cliente'))))

#seleciona os campos e maqueia os campos
clientes = clientes.alias('clie').join(contrato_id.alias('id'), 
                              (col('clie.cliente') == col('id.cliente'))
                              & (col('clie.valor_contrato_anual') == col('id.valor_contrato_anual'))
                              )  \
                  .select(col('id_contrato'), col('clie.valor_contrato_anual'), 'quantidade_de_servicos', 'cargo_responsavel', 'cep', 'data_inicio_contrato', 'nivel_de_importancia')
                  

##Zona de Insights e Data Visualization

###Insight 1

In [0]:
#Códigos

In [0]:
funcionarios.alias('func').join(cargos.alias('c'), ['cargo'], 'inner') \
            .filter(col('data_de_demissao').isNull()) \
            .agg(countDistinct(col('func.id_rh')).alias('qtde_funcionarios')) \
    .display()

qtde_funcionarios
436


Output can only be rendered in Databricks

In [0]:
funcionarios.alias('func').join(cargos.alias('c'), ['cargo'], 'inner') \
            .filter(col('data_de_demissao').isNull()) \
            .groupBy(col('c.area')) \
            .agg(countDistinct(col('func.id_rh')).alias('qtde_funcionarios')) \
            .sort(col('c.area')) \
    .display()

area,qtde_funcionarios
Administrativo,95
Comercial,100
Financeiro,73
Logísitca,74
Operações,94


Output can only be rendered in Databricks

In [0]:
funcionarios.alias('func').join(cargos.alias('c'), ['cargo'], 'inner') \
            .filter(col('data_de_demissao').isNull()) \
            .groupBy(col('c.nivel'), col('c.area'), col('c.quadro')) \
            .agg(countDistinct(col('func.id_rh')).alias('qtde_funcionarios')) \
            .sort(col('c.nivel'), col('c.area'), col('c.quadro')) \
    .display()

nivel,area,quadro,qtde_funcionarios
Analista,Administrativo,Terceiro,27
Analista,Comercial,Terceiro,28
Analista,Financeiro,Terceiro,23
Analista,Logísitca,Terceiro,30
Analista,Operações,Terceiro,21
Coordenador,Administrativo,Terceiro,24
Coordenador,Comercial,Terceiro,22
Coordenador,Financeiro,Terceiro,13
Coordenador,Logísitca,Terceiro,18
Coordenador,Operações,Terceiro,19


Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks

Em uma empresa onde temos 436 colabores, podemos notar que a área de Operações é a área onde se concentra a maior quantidade de estágiarios e diretores, e na Logistica é a área que é mais composta por analistas.

Link do DashBoard:
https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/1354057715866374/4090921080579626/4574391252210675/latest.html

###Insight 2

In [0]:
#areas dos funcionarios ativos
funcionarios.alias('func').join(cargos.alias('c'), ['cargo'], 'inner') \
  .filter(col('data_de_demissao').isNull()) \
  .groupBy(col('c.nivel'), col('c.area'), col('c.quadro')) \
  .agg(
      countDistinct(col('func.id_rh')).alias('qtde_funcionarios'),
      sum(col('salario_base')).alias('salarios'),
      sum(col('impostos')).alias('impostos'),
      sum(col('beneficios')).alias('beneficios'),
      sum(col('vt')).alias('vt'),
      sum(col('vr')).alias('vr') 
       ) \
  .sort(col('c.nivel'), col('c.area'), col('c.quadro')) \
  .display()
 

nivel,area,quadro,qtde_funcionarios,salarios,impostos,beneficios,vt,vr
Analista,Administrativo,Terceiro,27,196910.5,196910.5,78764.2,6116.0,18443.04
Analista,Comercial,Terceiro,28,183180.0,183180.0,73271.99999999999,6512.0,18527.04
Analista,Financeiro,Terceiro,23,133755.5,133755.5,53502.2,5544.0,15274.560000000001
Analista,Logísitca,Terceiro,30,187138.5,187138.5,74855.40000000001,7788.0,19360.32
Analista,Operações,Terceiro,21,112927.0,112927.0,45170.8,4818.0,14118.72
Coordenador,Administrativo,Terceiro,24,159064.5,159064.5,63625.79999999999,5346.0,15288.000000000002
Coordenador,Comercial,Terceiro,22,134043.5,134043.5,53617.399999999994,4906.0,14676.480000000003
Coordenador,Financeiro,Terceiro,13,77166.5,77166.5,30866.6,2816.0,8786.4
Coordenador,Logísitca,Terceiro,18,125792.5,125792.5,50317.00000000001,4290.0,12677.28
Coordenador,Operações,Terceiro,19,139468.0,139468.0,55787.200000000004,4466.0,13698.720000000003


Output can only be rendered in Databricks

Podemos notar que a área de Operações por ter mais estágiarios que qualquer outra área, é onde se tem um maior gasto em pagamento de salário

###Insight 3

In [0]:
#regiao dos funcionarios ativos
funcionarios.alias('func').join(cep.alias('cep'), ['cep'], 'inner') \
  .filter(col('data_de_demissao').isNull()) \
  .groupBy(col('cep.regiao')) \
  .agg(countDistinct(col('func.id_rh')).alias('qtde_funcionarios'),
       sum('salario_base').alias('salarios')) \
  .sort(col('cep.regiao')) \
  .display()

regiao,qtde_funcionarios,salarios
Centro - Oeste,58,350083.5
Nordeste,132,884550.0
Norte,131,792781.0
Sudeste,66,426373.5
Sul,49,344787.0


Output can only be rendered in Databricks

Notamos que a região do Nordeste e Norte são onde se gasta mais com salários

###Insight 4

In [0]:
#nivel de importancia dos contratos e valor de venda
#nivel_importancia_regiao = 
clientes.alias('clie').join(cep.alias('cep'), ['cep'], 'inner') \
  .agg(sum(col('clie.valor_contrato_anual')).alias('valor_contrato'), 
       countDistinct('id_contrato').alias("qtde_contratos"), 
       sum('quantidade_de_servicos').alias('qtde_servicos')) \
  .display()

valor_contrato,qtde_contratos,qtde_servicos
7.85295E7,311,17358.0


Output can only be rendered in Databricks

Output can only be rendered in Databricks

In [0]:
#nivel de importancia dos contratos e valor de venda
#nivel_importancia_regiao = 
clientes.alias('clie').join(cep.alias('cep'), ['cep'], 'inner') \
  .groupBy(col('cep.regiao'), col('clie.nivel_de_importancia')) \
  .agg(sum(col('clie.valor_contrato_anual')).alias('valor_contrato'), 
       countDistinct('id_contrato').alias("qtde_contratos"), 
       sum('quantidade_de_servicos').alias('qtde_servicos')) \
  .sort(col('cep.regiao')) \
  .display()

regiao,nivel_de_importancia,valor_contrato,qtde_contratos,qtde_servicos
Centro - Oeste,3,2344500.0,8,487.0
Centro - Oeste,2,2362500.0,8,346.0
Centro - Oeste,4,3366000.0,11,793.0
Centro - Oeste,1,792000.0,6,368.0
Nordeste,4,6880500.0,29,1557.0
Nordeste,2,5211000.0,18,852.0
Nordeste,3,8302500.0,29,1947.0
Nordeste,1,5121000.0,24,1366.0
Norte,2,3438000.0,16,841.0
Norte,4,7263000.0,28,1427.0


Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks

Nestes gráficos, conseguimos entender o porque as regiões Norte e Nordeste gastam mais em salários, pois são as regiões que mais tem contratos e serviços. Para termos uma dimensão: De 17.358 serviços, 64% correspondem as respectivas regiões


Link Dashnoad Contratos: https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/1354057715866374/4090921080579626/4574391252210675/latest.html

###Insight 5

In [0]:
clientes.groupBy(date_trunc('month',col('data_inicio_contrato')).alias('data'), 'nivel_de_importancia') \
  .agg(sum('valor_contrato_anual').alias('valor'), 
       sum('quantidade_de_servicos').alias('qtde_servicos')) \
  .sort('data', 'nivel_de_importancia' ) \
  .display()

data,nivel_de_importancia,valor,qtde_servicos
2018-10-01T00:00:00.000+0000,1,1467000.0,273.0
2018-10-01T00:00:00.000+0000,2,229500.0,58.0
2018-10-01T00:00:00.000+0000,3,1278000.0,445.0
2018-10-01T00:00:00.000+0000,4,1561500.0,272.0
2018-11-01T00:00:00.000+0000,1,2133000.0,645.0
2018-11-01T00:00:00.000+0000,2,2862000.0,582.0
2018-11-01T00:00:00.000+0000,3,2929500.0,660.0
2018-11-01T00:00:00.000+0000,4,2763000.0,379.0
2018-12-01T00:00:00.000+0000,1,1710000.0,332.0
2018-12-01T00:00:00.000+0000,2,3802500.0,949.0


Output can only be rendered in Databricks

Dentro da base de contratos, podemos observar que Janeiro/2019 foi o melhor mês para serviços de Importância Nível 3 em relação a valor de contrato. Em Abril/2019 os serviços de níveis 1, 3 e 4 tiveram uma queda depois de 5 meses de crescimento/estabilidade, neste mês em questão, o único que teve crescimento foi do nível 2.